In [ ]:
# default_exp binner
%load_ext autoreload
%autoreload 2

# binner

binning

In [ ]:
#hide
import numpy as np
from nbdev.showdoc import *
from jupydoc import NBdevCell
!rm -f images/fig_*.*

In [ ]:
#export
import numpy as np
import pandas as pd
class BinnedWeights(object):
    """ Generate a list of cells, with access to cell data
        weights
    """
    
    def __init__(self, 
                    data:'WeightedData object, with edges, source_name', 
                    bins:''=None):
        """
        Use time binning and data (a TimedData) object to generate list of cells
        """
        self.data = data 
        self.source_name = data.source_name
        self.verbose = getattr(data, 'verbose', 1)
        
        if bins is None:
            # get predefined bin data and corresponding fractional exposure
            # (this case using default step, range) 
            bins = data.edges
        elif np.isscalar(bins):
            # scalar is step
            step = bins
            tstart = data.edges[0]
            time_range = data.edges[-1]-tstart
            nbins = int(time_range/step)+1; 
            if self.verbose>0:
                print(f'Selecting {nbins} intervals of {step} days')
            bins = tstart + np.arange(nbins+1)*step 
        else:
            pass # check that bins makes sense:
        # the exposure array already part of of a WeightedDataX object
        exposure =  data.get_binned_exposure(bins) 
        self.bins=bins
        self.N = len(bins)-1 # number of bins
        self.bin_centers = 0.5*(self.bins[1:]+self.bins[:-1])
        self.fexposure = exposure/np.sum(exposure)   

        # get the photon data with good weights, not NaN
        if 'weight' in data.photon_data:
            w = data.photon_data.weight
            good = np.logical_not(np.isnan(w))
            self.photons = data.photon_data.loc[good]
            self.weights = w = self.photons.weight.values
            # estimates for total signal and background
            self.S = np.sum(w)
            self.B = np.sum(1-w)
        else:
            # no weights specified
            self.photons = data.photon_data
            self.S = self.B = 0
            self.weights=[]
            
        # use photon times to get indices of bin edges
        self._edges = np.searchsorted(self.photons.time, self.bins)
        
        
    def __repr__(self):
        return f'''{self.__class__}:  
        {len(self.fexposure)} intervals from {self.bins[0]:.1f} to {self.bins[-1]:.1f} for source {self.source_name}
        S {self.S:.2f}  B {self.B:.2f} '''

    def __getitem__(self, i):
        """ get info for ith time bin and return dict with 
            t : MJD
            tw: bin width, 
            fexp: exposure as fraction of total, 
            n : number of photons in bin
            w : weights 
            S,B:  value
        """
        k   = self._edges        
        if len(self.weights)==0: 
            # No weights
            wts = []
            n = len(self.photons[k[i]:k[i+1]])
        else:
            wts = self.weights[k[i]:k[i+1]]
            n = len(wts)
        exp = self.fexposure[i]
        tw  = self.bins[i+1]-self.bins[i]

        return dict(
                t=self.bin_centers[i], # time
                tw = tw,  # bin width
                fexp=exp*self.N, # exposure as a fraction of mean, for filtering
                n=n, # number of photons in bin
                w=wts,
                S= exp*self.S,
                B= exp*self.B,               
                )

    def __len__(self):
        return self.N
    
    @property
    def dataframe(self):
        """ a view of the data as a DataFrame, using the MJD time as an index
        """
        d = dict()
        for cell in self:
            d[cell['t']] = cell
        return pd.DataFrame.from_dict(d, orient='index' )

    def test_plots(self):
        """Make a set of plots of exposure, counts, properties of weights, if any
        """
        import matplotlib.pyplot as plt

        has_weights = len(self.weights)>0
        fig, axx = plt.subplots( 5 if has_weights else 3, 1, 
                    figsize=(12,10 if has_weights else 6), 
                    sharex=True,
                    gridspec_kw=dict(hspace=0,top=0.95),)
        times=[]; vals = []

        for cell in self:
            t, e, n, w = [cell[q] for q in 't fexp n w'.split()]
            if e==0:
                continue
            times.append(t)
            v =  [e, n, n/e ]
            if has_weights:
                v= v + [ w.mean(), np.sum(w**2)/sum(w)]
            vals.append(v)
        vals = np.array(vals).T
        labels =  ['rel exp','counts','count rate']
        if has_weights: labels = labels +  ['mean weight', 'rms/mean weight']
        for ax, v, ylabel in zip(axx, vals,labels):
            ax.plot(times, v, '+b')
            ax.set(ylabel=ylabel)
            ax.grid(alpha=0.5)
        axx[-1].set(xlabel='MJD')
        fig.suptitle(self.source_name)

<h4 id="UTC" class="doc_header"><code>UTC</code><a href="__main__.py#L14" class="source_link" style="float:right">[source]</a></h4>

> <code>UTC</code>(**`mjd`**)

convert MJD value to ISO date string

<h4 id="MJD" class="doc_header"><code>MJD</code><a href="__main__.py#L11" class="source_link" style="float:right">[source]</a></h4>

> <code>MJD</code>(**`met`**)

convert MET to MJD

'2001-01-01 00:01'

In [ ]:
from light_curves.effective_area import EffectiveArea
from utilities import keyword_options

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()
!date

Converted 00_poisson.ipynb.
Converted 01_effective_area.ipynb.
Converted 02_data.ipynb.
Converted index.ipynb.
Sun Nov 29 12:30:18 PST 2020
